In [ ]:
import torch
import torch.nn as nn
import torch.quantization
import matplotlib.pyplot as plt
from train_test import load_checkpoint,test, train, train_kd, test_batch
from data_loader import get_data_loader
from vit import ViT
from vit_CNNFF import ViTCNNFF
from helpers import count_parameters, quantize_model
import time

import torch.ao.quantization.quantize_fx as quantize_fx
import onnx


import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [ ]:
torch.manual_seed(42)
train_loader, test_loader = get_data_loader(
    80, 2, "datasets/cifar-100/cifar-100-python", download=True
)

In [ ]:
model = ViTCNNFF(
    image_size=32,
    patch_size=4,
    num_classes=100,
    dim=64,
    depth=3,
    heads=4,
    mlp_dim=128,
    dropout=0.1,
).to("cpu")

model_save_path = "save_model/cifar-100/vit16_CNNFF"

model_load_path = f"{model_save_path}/best_model.pt"

load_checkpoint(model, model_load_path)

In [ ]:
# train(
#     model,
#     train_loader,
#     test_loader,
#     epochs=5,
#     learning_rate=0.001,
#     device="cuda",
#     pruning_method="structured",
#     weight_decay=0.0005,
# )

In [ ]:
# model = ViT(
#     image_size=32,
#     patch_size=4,
#     num_classes=100,
#     dim=128,
#     depth=3,
#     heads=3,
#     mlp_dim=256,
#     dropout=0.1,
# ).to("cpu")



In [ ]:
model.to("cpu")
model.eval()
backend = "x86"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized = torch.quantization.prepare(model, inplace=False)
model_static_quantized = torch.quantization.convert(
    model_static_quantized, inplace=False
)
model_static_quantized.to("cpu")

In [ ]:
batch_array = []

for batch in test_loader:
    batch_array.append(batch)

In [7]:
inference_time_pruned = 0
start_time = time.time()
for _ in range(20):
    for image_batch in batch_array:
        with torch.no_grad():
            test_batch(model_static_quantized, image_batch[0], device="cpu")
    inference_time_pruned += time.time() - start_time
print(f"model_quantized_time: {inference_time_pruned} ")

NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at ../aten/src/ATen/core/MetaFallbackKernel.cpp:23